In [ ]:

!pip install openai pandas openpyxl tqdm --quiet

In [2]:
# ============================================================
# ✅ GPT 감정분석 초고속 안정 버전 (Batch + JSON 출력 + 캐싱 + 길이보정)
# ============================================================



import pandas as pd
import json, time, re
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """
    여러 문장을 batch 단위로 GPT에 전달하여 감정 분석 수행
    중복 문장은 캐시(cache)에서 불러옴
    """
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="GPT 감정분석 중"):
        batch = texts[i:i + batch_size]

        # ✅ 1. 캐시에 없는 문장만 추출
        new_texts = [t for t in batch if t not in cache]

        # ✅ 2. 모두 캐시에 있다면 바로 append
        if not new_texts:
            for t in batch:
                results.append(cache[t])
            continue

        # ✅ 3. GPT 프롬프트 (JSON 출력 강제)
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장들의 감정을 JSON 배열로 정확히 출력해.
        각 문장은 반드시 부정/중립/긍정 중 하나의 감정(sentiment)과 신뢰도(confidence, 0~1)를 포함해야 해.

        출력 형식 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # ⚡ 초고속 모델
                messages=[
                    {"role": "system", "content": "너는 감정 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()

            # ✅ 4. JSON 파싱 시도
            json_match = re.search(r"\[.*\]", output, re.S)
            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 백업 파서로 재시도")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            # ✅ 5. batch_results 길이 보정 (핵심 추가)
            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 시 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                batch_results = (batch_results + 
                                 [{"sentiment": "중립", "confidence": 0.0}] * (len(new_texts) - len(batch_results)))[:len(new_texts)]

            # ✅ 6. 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            # ✅ 7. 전체 결과 병합
            for t in batch:
                results.append(cache[t])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            for t in batch:
                results.append({"sentiment": "중립", "confidence": 0.0})

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 데이터 로드
# ------------------------------------------------------------
input_paths = [
    r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\롯데카드_정보유출_comments_20251103_165555.xlsx",
    # r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\SKT_유심칩_유출_comments_20251103_165204.xlsx",
    # r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\KT_펨토셀_해킹_comments_20251103_152106.xlsx"
]

dfs = [pd.read_excel(path) for path in input_paths]
df = pd.concat(dfs, ignore_index=True)

if "text" not in df.columns:
    raise ValueError("⚠️ 입력 파일에 'text' 컬럼이 필요합니다!")

df["text"] = df["text"].astype(str).str.strip().str[:300]  # 과도한 길이 제한

# ------------------------------------------------------------
# 3️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["text"].tolist(), batch_size=20)

df["gpt_sentiment"] = [r["sentiment"] for r in results]
df["gpt_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 4️⃣ 결과 저장 (엑셀)
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\롯데카드__댓글감정분석_GPT_labeled.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ 분석 완료! 결과 저장 경로:\n📂 {output_path}")

# 상위 5행 미리보기
display(df.head())


GPT 감정분석 중: 100%|██████████| 271/271 [50:19<00:00, 11.14s/it] 



✅ 분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\롯데카드_댓글감정분석_GPT_labeled.xlsx


,videoId,author,text,publishedAt,gpt_sentiment,gpt_confidence
0,VaBG145v0UM,@newskbs,0:19 자막에 297만 여명을 2천 97만 여명으로 잘못 표기했습니다. 이용에 불...,2025-09-21T02:21:37Z,부정,0.85
1,VaBG145v0UM,@부정선거out,전세계의 악 ㅡ이슬람 중국,2025-10-23T13:06:36Z,부정,0.90
2,VaBG145v0UM,@보노_보더,아직으으으응으은?,2025-09-27T13:33:33Z,중립,0.50
3,VaBG145v0UM,@Ralph.C-h3r,껌이나 ㅊ ㅕ만들어가 죠떼야,2025-09-27T02:12:52Z,중립,0.55
4,VaBG145v0UM,@hejin_9158,지금은 여기저기서 떠드니까 해커들이 가져간 정보는 조용히 있겠지만 사그러들고 조용해...,2025-09-26T15:27:55Z,중립,0.50


In [3]:
# ============================================================
# ✅ GPT 감정분석 초고속 안정 버전 (Batch + JSON 출력 + 캐싱 + 길이보정)
# ============================================================



import pandas as pd
import json, time, re
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """
    여러 문장을 batch 단위로 GPT에 전달하여 감정 분석 수행
    중복 문장은 캐시(cache)에서 불러옴
    """
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="GPT 감정분석 중"):
        batch = texts[i:i + batch_size]

        # ✅ 1. 캐시에 없는 문장만 추출
        new_texts = [t for t in batch if t not in cache]

        # ✅ 2. 모두 캐시에 있다면 바로 append
        if not new_texts:
            for t in batch:
                results.append(cache[t])
            continue

        # ✅ 3. GPT 프롬프트 (JSON 출력 강제)
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장들의 감정을 JSON 배열로 정확히 출력해.
        각 문장은 반드시 부정/중립/긍정 중 하나의 감정(sentiment)과 신뢰도(confidence, 0~1)를 포함해야 해.

        출력 형식 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # ⚡ 초고속 모델
                messages=[
                    {"role": "system", "content": "너는 감정 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()

            # ✅ 4. JSON 파싱 시도
            json_match = re.search(r"\[.*\]", output, re.S)
            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 백업 파서로 재시도")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            # ✅ 5. batch_results 길이 보정 (핵심 추가)
            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 시 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                batch_results = (batch_results + 
                                 [{"sentiment": "중립", "confidence": 0.0}] * (len(new_texts) - len(batch_results)))[:len(new_texts)]

            # ✅ 6. 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            # ✅ 7. 전체 결과 병합
            for t in batch:
                results.append(cache[t])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            for t in batch:
                results.append({"sentiment": "중립", "confidence": 0.0})

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 데이터 로드
# ------------------------------------------------------------
input_paths = [
    r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\SKT_유심칩_유출_comments_20251103_165204.xlsx",
    # r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\SKT_유심칩_유출_comments_20251103_165204.xlsx",
    # r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\KT_펨토셀_해킹_comments_20251103_152106.xlsx"
]

dfs = [pd.read_excel(path) for path in input_paths]
df = pd.concat(dfs, ignore_index=True)

if "text" not in df.columns:
    raise ValueError("⚠️ 입력 파일에 'text' 컬럼이 필요합니다!")

df["text"] = df["text"].astype(str).str.strip().str[:300]  # 과도한 길이 제한

# ------------------------------------------------------------
# 3️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["text"].tolist(), batch_size=20)

df["gpt_sentiment"] = [r["sentiment"] for r in results]
df["gpt_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 4️⃣ 결과 저장 (엑셀)
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\SKT_댓글감정분석_GPT_labeled.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ 분석 완료! 결과 저장 경로:\n📂 {output_path}")

# 상위 5행 미리보기
display(df.head())


GPT 감정분석 중: 100%|██████████| 364/364 [1:04:12<00:00, 10.58s/it]



✅ 분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\SKT_댓글감정분석_GPT_labeled.xlsx


,videoId,author,text,publishedAt,gpt_sentiment,gpt_confidence
0,3B6nv74tSkw,@baramdori7,미국에 짓는 공장도 다 중단하고 그간 들어간 돈은 버렸다 생각하는게 이득이다. \n...,2025-10-11T10:35:14Z,부정,0.95
1,3B6nv74tSkw,@berrycherry6558,통신사 앱이라는게 SK텔레콤이면 SK텔레콤 앱을 깔아야 하는건가요?,2025-10-02T01:47:26Z,중립,0.60
2,3B6nv74tSkw,@이정란-g4r,KT앱에서 정보보호 알림이 안뜨는데요?,2025-09-27T02:06:02Z,중립,0.55
3,3B6nv74tSkw,@보라-j1w,회사 직원분이 보이스피싱 전화오셔서 대처 후 이 영상 보여드리고 따라하셨습니다 좋은...,2025-09-26T03:07:35Z,중립,0.50
4,3B6nv74tSkw,@아름드리-q5e,감사합니다.,2025-09-22T04:45:44Z,긍정,0.90


In [6]:
# ============================================================
# ✅ GPT 게시글 감정분석 (Batch + JSON + 캐싱 + 길이보정)
# ============================================================

import pandas as pd
import json, time, re
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """
    여러 문장을 batch 단위로 GPT에 전달하여 감정 분석 수행
    중복 문장은 캐시(cache)에서 불러옴
    """
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="📊 GPT 게시글 감정분석 중"):
        batch = texts[i:i + batch_size]

        # ✅ 1. 캐시에 없는 문장만 추출
        new_texts = [t for t in batch if t not in cache]

        # ✅ 2. 모두 캐시에 있다면 바로 append
        if not new_texts:
            for t in batch:
                results.append(cache[t])
            continue

        # ✅ 3. GPT 프롬프트 (JSON 출력 강제)
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장(게시글의 제목과 설명)의 전체 감정을 JSON 배열로 출력해.
        각 문장은 반드시 '긍정', '부정', '중립' 중 하나의 감정(sentiment)과
        신뢰도(confidence, 0~1)를 포함해야 해.

        출력 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "너는 게시글의 감정을 평가하는 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()

            # ✅ 4. JSON 파싱 시도
            json_match = re.search(r"\[.*\]", output, re.S)
            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 백업 파서로 재시도")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            # ✅ 5. batch_results 길이 보정 (핵심)
            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 시 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                batch_results = (batch_results + 
                                 [{"sentiment": "중립", "confidence": 0.0}] * (len(new_texts) - len(batch_results)))[:len(new_texts)]

            # ✅ 6. 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            # ✅ 7. 전체 결과 병합
            for t in batch:
                results.append(cache[t])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            for t in batch:
                results.append({"sentiment": "중립", "confidence": 0.0})

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 게시글 데이터 불러오기
# ------------------------------------------------------------
input_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\GS리테일_정보유출_videos_20251103_154830.xlsx"
df = pd.read_excel(input_path)

# ------------------------------------------------------------
# 3️⃣ 게시글 텍스트 결합 (제목 + 설명)
# ------------------------------------------------------------
df["text_raw"] = (
    df.get("title", "").astype(str).fillna("") + " " + df.get("description", "").astype(str).fillna("")
).str.strip().str[:500]  # 과도한 길이 제한 (500자)

print("총 게시글 수:", len(df))

# ------------------------------------------------------------
# 4️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["text_raw"].tolist(), batch_size=20)

# ------------------------------------------------------------
# 5️⃣ 결과 반영
# ------------------------------------------------------------
df["gpt_post_sentiment"] = [r["sentiment"] for r in results]
df["gpt_post_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 6️⃣ 결과 저장
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\GS_posts_GPT_labeled.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ 게시글 감정분석 완료! 결과 저장 경로:\n📂 {output_path}")

display(df.head())


총 게시글 수: 22


📊 GPT 게시글 감정분석 중: 100%|██████████| 2/2 [00:13<00:00,  6.80s/it]


✅ 게시글 감정분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\GS_posts_GPT_labeled.xlsx


,videoId,title,description,channelTitle,channelId,publishedAt,url,viewCount,likeCount,commentCount,subscriberCount,text_raw,gpt_post_sentiment,gpt_post_confidence
0,9GukLzTRkkM,"주소에 통관번호까지…GS리테일, 158만명 정보 유출 확인 / JTBC 뉴스룸",지난달 GS리테일에서도 9만 명의 개인정보가 유출되는 사건이 벌어졌습니다. 그런데 ...,JTBC News,UCsU-I-vHLiaMfV_ceaYz5rQ,2025-02-27T11:45:38Z,https://www.youtube.com/watch?v=9GukLzTRkkM,13151,228,120,4750000,"주소에 통관번호까지…GS리테일, 158만명 정보 유출 확인 / JTBC 뉴스룸 지난...",부정,0.95
1,qcQjIfuls3c,GS리테일 고객 정보 9만여 건 해킹 피해…3일 만에 알렸다 / SBS / #D리포트,GS리테일이 고객들에게 오늘(6일) 오후 발송한 해킹 피해 공지 문잡니다. 지난달 ...,SBS 뉴스,UCkinYTS9IHqOEwR1Sze2JTw,2025-01-06T09:57:34Z,https://www.youtube.com/watch?v=qcQjIfuls3c,6297,68,42,5080000,GS리테일 고객 정보 9만여 건 해킹 피해…3일 만에 알렸다 / SBS / #D리포...,부정,0.92
2,ghSztNUxjB8,GS리테일 고객 9만 명 개인정보 유출…해킹 공격 / SBS 8뉴스,GS리테일이 지난달 27일부터 이달 4일 사이 웹사이트 해킹 공격을 받아 고객 9만...,SBS 뉴스,UCkinYTS9IHqOEwR1Sze2JTw,2025-01-06T12:38:57Z,https://www.youtube.com/watch?v=ghSztNUxjB8,4276,28,10,5080000,GS리테일 고객 9만 명 개인정보 유출…해킹 공격 / SBS 8뉴스 GS리테일이 지...,부정,0.93
3,dAGkZUtGJ7Y,"GS리테일, 편의점에 홈쇼핑도 해킹‥158만 건 추가 유출 (2025.02.27/뉴...",지난달 편의점 홈페이지에서 9만여 명의 개인정보가 유출된 GS리테일이 홈쇼핑 웹사이...,MBCNEWS,UCF4Wxdo3inmxP-Y59wXDsFw,2025-02-27T06:24:05Z,https://www.youtube.com/watch?v=dAGkZUtGJ7Y,3362,65,16,6040000,"GS리테일, 편의점에 홈쇼핑도 해킹‥158만 건 추가 유출 (2025.02.27/뉴...",부정,0.94
4,UwvL4geTjuA,GS리테일 고객 9만명 해킹..&#39;겨울별미&#39; 대구 씨 말랐다 (이슈라이...,겨울철 생선인 대구 어획량이 크게 줄어들었습니다. 지난해의 3분의 1도 미치지 못하...,SBS 뉴스,UCkinYTS9IHqOEwR1Sze2JTw,2025-01-07T02:43:45Z,https://www.youtube.com/watch?v=UwvL4geTjuA,3027,10,0,5080000,GS리테일 고객 9만명 해킹..&#39;겨울별미&#39; 대구 씨 말랐다 (이슈라이...,부정,0.91


In [7]:
# ============================================================
# ✅ GPT 게시글 감정분석 (Batch + JSON + 캐싱 + 길이보정)
# ============================================================

import pandas as pd
import json, time, re
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """
    여러 문장을 batch 단위로 GPT에 전달하여 감정 분석 수행
    중복 문장은 캐시(cache)에서 불러옴
    """
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="📊 GPT 게시글 감정분석 중"):
        batch = texts[i:i + batch_size]

        # ✅ 1. 캐시에 없는 문장만 추출
        new_texts = [t for t in batch if t not in cache]

        # ✅ 2. 모두 캐시에 있다면 바로 append
        if not new_texts:
            for t in batch:
                results.append(cache[t])
            continue

        # ✅ 3. GPT 프롬프트 (JSON 출력 강제)
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장(게시글의 제목과 설명)의 전체 감정을 JSON 배열로 출력해.
        각 문장은 반드시 '긍정', '부정', '중립' 중 하나의 감정(sentiment)과
        신뢰도(confidence, 0~1)를 포함해야 해.

        출력 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "너는 게시글의 감정을 평가하는 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()

            # ✅ 4. JSON 파싱 시도
            json_match = re.search(r"\[.*\]", output, re.S)
            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 백업 파서로 재시도")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            # ✅ 5. batch_results 길이 보정 (핵심)
            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 시 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                batch_results = (batch_results + 
                                 [{"sentiment": "중립", "confidence": 0.0}] * (len(new_texts) - len(batch_results)))[:len(new_texts)]

            # ✅ 6. 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            # ✅ 7. 전체 결과 병합
            for t in batch:
                results.append(cache[t])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            for t in batch:
                results.append({"sentiment": "중립", "confidence": 0.0})

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 게시글 데이터 불러오기
# ------------------------------------------------------------
input_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\KT_펨토셀_해킹_videos_20251103_152106.xlsx"
df = pd.read_excel(input_path)

# ------------------------------------------------------------
# 3️⃣ 게시글 텍스트 결합 (제목 + 설명)
# ------------------------------------------------------------
df["text_raw"] = (
    df.get("title", "").astype(str).fillna("") + " " + df.get("description", "").astype(str).fillna("")
).str.strip().str[:500]  # 과도한 길이 제한 (500자)

print("총 게시글 수:", len(df))

# ------------------------------------------------------------
# 4️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["text_raw"].tolist(), batch_size=20)

# ------------------------------------------------------------
# 5️⃣ 결과 반영
# ------------------------------------------------------------
df["gpt_post_sentiment"] = [r["sentiment"] for r in results]
df["gpt_post_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 6️⃣ 결과 저장
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\KT_posts_GPT_labeled.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ 게시글 감정분석 완료! 결과 저장 경로:\n📂 {output_path}")

display(df.head())


총 게시글 수: 349


📊 GPT 게시글 감정분석 중: 100%|██████████| 18/18 [02:42<00:00,  9.04s/it]


✅ 게시글 감정분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\KT_posts_GPT_labeled.xlsx


,videoId,title,description,channelTitle,channelId,publishedAt,url,viewCount,likeCount,commentCount,subscriberCount,text_raw,gpt_post_sentiment,gpt_post_confidence
0,F0UQQT0tSD8,결제 막히자 1분 뒤..KT 해커가 직접 쓴 글 &#39;충격&#39; #뉴스다 /...,"0:00 피해자들 '그곳' 지나다녔다…소액결제 사태 ""더 큰 공격 대비해야"" (9....",JTBC News,UCsU-I-vHLiaMfV_ceaYz5rQ,2025-09-11T01:09:51Z,https://www.youtube.com/watch?v=F0UQQT0tSD8,1291343,9573,3459,4750000,결제 막히자 1분 뒤..KT 해커가 직접 쓴 글 &#39;충격&#39; #뉴스다 /...,부정,0.95
1,4eqeGxSyCts,"&quot;아~~어떡해!&quot; KT어플 들어갔다 &#39;경악&#39;, 광명...",최근 서울 금천구와 경기도 광명시에서 KT 이용자의 휴대전화에서 소액 결제로 수십만...,MBCNEWS,UCF4Wxdo3inmxP-Y59wXDsFw,2025-09-10T06:56:21Z,https://www.youtube.com/watch?v=4eqeGxSyCts,888588,6269,1002,6040000,"&quot;아~~어떡해!&quot; KT어플 들어갔다 &#39;경악&#39;, 광명...",부정,0.92
2,_QuSUseohg0,"[자막뉴스] 알아차린 카이스트, KT에 알렸다…뒤늦게 밝혀진 사실 / JTBC News","이번 사태는 전문가들이 10년 전부터 우려했던 일입니다. 2014년, 국내 한 연구...",JTBC News,UCsU-I-vHLiaMfV_ceaYz5rQ,2025-09-13T02:00:29Z,https://www.youtube.com/watch?v=_QuSUseohg0,817923,6693,1321,4750000,"[자막뉴스] 알아차린 카이스트, KT에 알렸다…뒤늦게 밝혀진 사실 / JTBC Ne...",부정,0.90
3,49Qf4b7bWMM,KT 대표 고개 숙였다…&quot;유심 전면 교체·전액 보상할 것&quot; / SBS,"〈앵커〉 최근 발생한 무단 소액결제 사태와 관련해, KT가 5천5백여 명의 개인정보...",SBS 뉴스,UCkinYTS9IHqOEwR1Sze2JTw,2025-09-11T08:27:46Z,https://www.youtube.com/watch?v=49Qf4b7bWMM,786671,3663,2886,5080000,KT 대표 고개 숙였다…&quot;유심 전면 교체·전액 보상할 것&quot; / S...,부정,0.93
4,JejEYG3CfsM,[단독] &quot;유령 기지국 만들어 KT망 해킹‥다른 지역으로 이동도 가능&qu...,최근 KT 휴대전화 가입자들로부터 소액 결제를 통해 수십만 원씩이 빠져나간 사건에 ...,MBCNEWS,UCF4Wxdo3inmxP-Y59wXDsFw,2025-09-09T10:59:40Z,https://www.youtube.com/watch?v=JejEYG3CfsM,650828,6807,2827,6040000,[단독] &quot;유령 기지국 만들어 KT망 해킹‥다른 지역으로 이동도 가능&qu...,부정,0.94


In [8]:
# ============================================================
# ✅ GPT 게시글 감정분석 (Batch + JSON + 캐싱 + 길이보정)
# ============================================================

import pandas as pd
import json, time, re
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """
    여러 문장을 batch 단위로 GPT에 전달하여 감정 분석 수행
    중복 문장은 캐시(cache)에서 불러옴
    """
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="📊 GPT 게시글 감정분석 중"):
        batch = texts[i:i + batch_size]

        # ✅ 1. 캐시에 없는 문장만 추출
        new_texts = [t for t in batch if t not in cache]

        # ✅ 2. 모두 캐시에 있다면 바로 append
        if not new_texts:
            for t in batch:
                results.append(cache[t])
            continue

        # ✅ 3. GPT 프롬프트 (JSON 출력 강제)
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장(게시글의 제목과 설명)의 전체 감정을 JSON 배열로 출력해.
        각 문장은 반드시 '긍정', '부정', '중립' 중 하나의 감정(sentiment)과
        신뢰도(confidence, 0~1)를 포함해야 해.

        출력 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "너는 게시글의 감정을 평가하는 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()

            # ✅ 4. JSON 파싱 시도
            json_match = re.search(r"\[.*\]", output, re.S)
            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 백업 파서로 재시도")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            # ✅ 5. batch_results 길이 보정 (핵심)
            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 시 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                batch_results = (batch_results + 
                                 [{"sentiment": "중립", "confidence": 0.0}] * (len(new_texts) - len(batch_results)))[:len(new_texts)]

            # ✅ 6. 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            # ✅ 7. 전체 결과 병합
            for t in batch:
                results.append(cache[t])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            for t in batch:
                results.append({"sentiment": "중립", "confidence": 0.0})

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 게시글 데이터 불러오기
# ------------------------------------------------------------
input_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\SKT_유심칩_유출_videos_20251103_165204.xlsx"
df = pd.read_excel(input_path)

# ------------------------------------------------------------
# 3️⃣ 게시글 텍스트 결합 (제목 + 설명)
# ------------------------------------------------------------
df["text_raw"] = (
    df.get("title", "").astype(str).fillna("") + " " + df.get("description", "").astype(str).fillna("")
).str.strip().str[:500]  # 과도한 길이 제한 (500자)

print("총 게시글 수:", len(df))

# ------------------------------------------------------------
# 4️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["text_raw"].tolist(), batch_size=20)

# ------------------------------------------------------------
# 5️⃣ 결과 반영
# ------------------------------------------------------------
df["gpt_post_sentiment"] = [r["sentiment"] for r in results]
df["gpt_post_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 6️⃣ 결과 저장
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\skt_posts_GPT_labeled.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ 게시글 감정분석 완료! 결과 저장 경로:\n📂 {output_path}")

display(df.head())


총 게시글 수: 500


📊 GPT 게시글 감정분석 중: 100%|██████████| 25/25 [03:43<00:00,  8.93s/it]



✅ 게시글 감정분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\skt_posts_GPT_labeled.xlsx


,videoId,title,description,channelTitle,channelId,publishedAt,url,viewCount,likeCount,commentCount,subscriberCount,text_raw,gpt_post_sentiment,gpt_post_confidence
0,3B6nv74tSkw,지금 유심이 문제가 아닙니다 이거부터 당장 끄세요! (모든 통신사 공통),1분미만 #꿀팁 #유심 #유출 #설정 #정보.,1분미만,UC2xkO7XCiStWfR3fKbzkbqw,2025-04-26T08:19:53Z,https://www.youtube.com/watch?v=3B6nv74tSkw,4730371,130601,5631,2570000,지금 유심이 문제가 아닙니다 이거부터 당장 끄세요! (모든 통신사 공통) 1분미만 ...,부정,0.95
1,6qYETkvx4cY,휴대폰 이거 안 끄면 다 털립니다. 빨리 끄세요!,내 개인정보도 혹시...? 채널에 가입하여 혜택을 누려보세요.,표영호 tv / 경제적 자유와 행복을 위하여,UCp8lZRI3NSCXxhn_a_WGVKQ,2025-04-28T10:30:06Z,https://www.youtube.com/watch?v=6qYETkvx4cY,3617269,67184,3981,810000,휴대폰 이거 안 끄면 다 털립니다. 빨리 끄세요! 내 개인정보도 혹시...? 채널에...,부정,0.92
2,yPFP6QcL_wI,최악의 사태와 대응.. SKT 유심 해킹&amp;피해 대처법 총정리,"SKT 해킹 사태가 터진 후 당일 소식과 초기 대응 방법을 커뮤니티에 공유했고, S...",ITSub잇섭,UCdUcjkyZtf-1WJyPPiETF1g,2025-04-28T10:30:42Z,https://www.youtube.com/watch?v=yPFP6QcL_wI,3256113,80039,10238,2780000,최악의 사태와 대응.. SKT 유심 해킹&amp;피해 대처법 총정리 SKT 해킹 사...,중립,0.60
3,P5UiGz68mZk,&quot;핸드폰 꺼놓는 순간&quot;…SKT폰 &#39;이건 하지 말아야&#39...,"0:00 '유심보호' 고객에 하라는 SKT…오픈런 해도 ""재고 없어요"" (4.28 ...",JTBC News,UCsU-I-vHLiaMfV_ceaYz5rQ,2025-04-29T00:55:01Z,https://www.youtube.com/watch?v=P5UiGz68mZk,2844108,19354,3419,4750000,&quot;핸드폰 꺼놓는 순간&quot;…SKT폰 &#39;이건 하지 말아야&#39...,부정,0.90
4,rlaKrI4bjQs,skt 유심해킹이후 대처법 빨리하세요!,완전 난리난 skt 유심해킹이후 대처법 빨리하세요!,만렙백수,UCn8tvVLoVaPP4MjDwMff5MA,2025-04-26T12:11:40Z,https://www.youtube.com/watch?v=rlaKrI4bjQs,5343517,91543,2318,89200,skt 유심해킹이후 대처법 빨리하세요! 완전 난리난 skt 유심해킹이후 대처법 빨리...,부정,0.93


In [9]:
# ============================================================
# ✅ GPT 게시글 감정분석 (Batch + JSON + 캐싱 + 길이보정)
# ============================================================

import pandas as pd
import json, time, re
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """
    여러 문장을 batch 단위로 GPT에 전달하여 감정 분석 수행
    중복 문장은 캐시(cache)에서 불러옴
    """
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="📊 GPT 게시글 감정분석 중"):
        batch = texts[i:i + batch_size]

        # ✅ 1. 캐시에 없는 문장만 추출
        new_texts = [t for t in batch if t not in cache]

        # ✅ 2. 모두 캐시에 있다면 바로 append
        if not new_texts:
            for t in batch:
                results.append(cache[t])
            continue

        # ✅ 3. GPT 프롬프트 (JSON 출력 강제)
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장(게시글의 제목과 설명)의 전체 감정을 JSON 배열로 출력해.
        각 문장은 반드시 '긍정', '부정', '중립' 중 하나의 감정(sentiment)과
        신뢰도(confidence, 0~1)를 포함해야 해.

        출력 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "너는 게시글의 감정을 평가하는 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()

            # ✅ 4. JSON 파싱 시도
            json_match = re.search(r"\[.*\]", output, re.S)
            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 백업 파서로 재시도")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            # ✅ 5. batch_results 길이 보정 (핵심)
            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 시 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                batch_results = (batch_results + 
                                 [{"sentiment": "중립", "confidence": 0.0}] * (len(new_texts) - len(batch_results)))[:len(new_texts)]

            # ✅ 6. 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            # ✅ 7. 전체 결과 병합
            for t in batch:
                results.append(cache[t])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            for t in batch:
                results.append({"sentiment": "중립", "confidence": 0.0})

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 게시글 데이터 불러오기
# ------------------------------------------------------------
input_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\롯데카드_정보유출_videos_20251103_165555.xlsx"
df = pd.read_excel(input_path)

# ------------------------------------------------------------
# 3️⃣ 게시글 텍스트 결합 (제목 + 설명)
# ------------------------------------------------------------
df["text_raw"] = (
    df.get("title", "").astype(str).fillna("") + " " + df.get("description", "").astype(str).fillna("")
).str.strip().str[:500]  # 과도한 길이 제한 (500자)

print("총 게시글 수:", len(df))

# ------------------------------------------------------------
# 4️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["text_raw"].tolist(), batch_size=20)

# ------------------------------------------------------------
# 5️⃣ 결과 반영
# ------------------------------------------------------------
df["gpt_post_sentiment"] = [r["sentiment"] for r in results]
df["gpt_post_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 6️⃣ 결과 저장
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\롯데데_posts_GPT_labeled.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ 게시글 감정분석 완료! 결과 저장 경로:\n📂 {output_path}")

display(df.head())


총 게시글 수: 500


📊 GPT 게시글 감정분석 중: 100%|██████████| 25/25 [04:49<00:00, 11.56s/it]



✅ 게시글 감정분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\롯데데_posts_GPT_labeled.xlsx


,videoId,title,description,channelTitle,channelId,publishedAt,url,viewCount,likeCount,commentCount,subscriberCount,text_raw,gpt_post_sentiment,gpt_post_confidence
0,VaBG145v0UM,"[자막뉴스] 롯데카드 쓰는데, &#39;이 문자&#39; 받았다면 당장 카드 해지하...","해킹은 지난달 14일 시작됐습니다. 악성코드로 온라인 결제 서버를 뚫었고, 해커들은...",KBS News,UCcQTRi69dsVYHN3exePtZ1A,2025-09-19T01:35:02Z,https://www.youtube.com/watch?v=VaBG145v0UM,638378,3443,875,3480000,"[자막뉴스] 롯데카드 쓰는데, &#39;이 문자&#39; 받았다면 당장 카드 해지하...",부정,0.95
1,4XwhXsUVHN0,충격적 사실! 외국 해커에게 완전히 뚫린 한국정부기관! [코너별 다시보기],매불쇼 #최욱 #롯데카드 #KT - 보안 특집 : 김승주 X 오윤혜 --------...,[팟빵] 매불쇼,UCMYhq9OyGI5UEz_NTAoHY7A,2025-09-22T11:42:16Z,https://www.youtube.com/watch?v=4XwhXsUVHN0,302503,8080,503,2830000,충격적 사실! 외국 해커에게 완전히 뚫린 한국정부기관! [코너별 다시보기] 매불쇼 ...,부정,0.92
2,rc87JZwNqFA,[단독] 롯데카드 해킹에서도…소름 돋는 공통점 찾았다 / SBS 8뉴스,"〈앵커〉 297만 명에 달하는 개인정보가 털린 롯데카드 해킹 사태가, 중국 해킹 조...",SBS 뉴스,UCkinYTS9IHqOEwR1Sze2JTw,2025-10-01T12:00:22Z,https://www.youtube.com/watch?v=rc87JZwNqFA,285226,7523,3774,5080000,[단독] 롯데카드 해킹에서도…소름 돋는 공통점 찾았다 / SBS 8뉴스 〈앵커〉 2...,부정,0.90
3,7Pr_Vvn-4Og,"[단독] &quot;LG U+, KT도 털렸다‥&#39;해킹 아냐&#39; 버티기에...","SK텔레콤에서 2천3백만 가입자 정보가 유출돼, 유심 대란이 벌어진 게 넉 달 전이...",MBCNEWS,UCF4Wxdo3inmxP-Y59wXDsFw,2025-09-01T11:50:17Z,https://www.youtube.com/watch?v=7Pr_Vvn-4Og,268457,3218,1215,6040000,"[단독] &quot;LG U+, KT도 털렸다‥&#39;해킹 아냐&#39; 버티기에...",부정,0.91
4,GnYiq1DhdtI,"[단독] 롯데카드, 부정사용 없다더니‥2차 &#39;피싱&#39; 공격 확인 (20...",297만 명의 개인정보가 유출된 롯데카드 해킹 사태의 파장이 확산되고 있습니다. 유...,MBCNEWS,UCF4Wxdo3inmxP-Y59wXDsFw,2025-09-19T10:53:51Z,https://www.youtube.com/watch?v=GnYiq1DhdtI,211262,2180,546,6040000,"[단독] 롯데카드, 부정사용 없다더니‥2차 &#39;피싱&#39; 공격 확인 (20...",부정,0.93


여기부터 다시하기 (11월 7일)

In [2]:
# ============================================================
# ✅ GPT 감정분석 초고속 안정 버전 (Batch + JSON 출력 + 캐싱 + 길이보정)
# ============================================================



import pandas as pd
import json, time, re
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """
    여러 문장을 batch 단위로 GPT에 전달하여 감정 분석 수행
    중복 문장은 캐시(cache)에서 불러옴
    """
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="GPT 감정분석 중"):
        batch = texts[i:i + batch_size]

        # ✅ 1. 캐시에 없는 문장만 추출
        new_texts = [t for t in batch if t not in cache]

        # ✅ 2. 모두 캐시에 있다면 바로 append
        if not new_texts:
            for t in batch:
                results.append(cache[t])
            continue

        # ✅ 3. GPT 프롬프트 (JSON 출력 강제)
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장들의 감정을 JSON 배열로 정확히 출력해.
        각 문장은 반드시 부정/중립/긍정 중 하나의 감정(sentiment)과 신뢰도(confidence, 0~1)를 포함해야 해.

        출력 형식 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # ⚡ 초고속 모델
                messages=[
                    {"role": "system", "content": "너는 감정 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()

            # ✅ 4. JSON 파싱 시도
            json_match = re.search(r"\[.*\]", output, re.S)
            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 백업 파서로 재시도")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            # ✅ 5. batch_results 길이 보정 (핵심 추가)
            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 시 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                batch_results = (batch_results + 
                                 [{"sentiment": "중립", "confidence": 0.0}] * (len(new_texts) - len(batch_results)))[:len(new_texts)]

            # ✅ 6. 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            # ✅ 7. 전체 결과 병합
            for t in batch:
                results.append(cache[t])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            for t in batch:
                results.append({"sentiment": "중립", "confidence": 0.0})

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 데이터 로드
# ------------------------------------------------------------
input_paths = [
    r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\KT_펨토셀_해킹_comments_20251103_152106.xlsx",
    # r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\SKT_유심칩_유출_comments_20251103_165204.xlsx",
    # r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\KT_펨토셀_해킹_comments_20251103_152106.xlsx"
]

dfs = [pd.read_excel(path) for path in input_paths]
df = pd.concat(dfs, ignore_index=True)

if "text" not in df.columns:
    raise ValueError("⚠️ 입력 파일에 'text' 컬럼이 필요합니다!")

df["text"] = df["text"].astype(str).str.strip().str[:300]  # 과도한 길이 제한

# ------------------------------------------------------------
# 3️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["text"].tolist(), batch_size=20)

df["gpt_sentiment"] = [r["sentiment"] for r in results]
df["gpt_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 4️⃣ 결과 저장 (엑셀)
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\KT_댓글감정분석_GPT_labeled.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ 분석 완료! 결과 저장 경로:\n📂 {output_path}")

# 상위 5행 미리보기
display(df.head())


GPT 감정분석 중: 100%|██████████| 327/327 [59:45<00:00, 10.97s/it]  



✅ 분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\KT_댓글감정분석_GPT_labeled.xlsx


,videoId,author,text,publishedAt,gpt_sentiment,gpt_confidence
0,F0UQQT0tSD8,@samelia-w1i,20 년전 대포폰 사건을 시작으로 폰 으로 인한 범죄는 가중될것이다,2025-10-22T10:37:31Z,부정,0.85
1,F0UQQT0tSD8,@종호박-g9c,2차 인증수단 바꿨을거 아니여 지금쯤 피해자 더 이상 안나오면 어느정도 답나온거 아...,2025-10-07T16:45:57Z,부정,0.78
2,F0UQQT0tSD8,@행복코리아,카톡으로도 가끔 주민번호 물어보는 설문지 뜨거나 수상한 흔적이 가끔 떠서 의문이 들...,2025-10-02T00:39:27Z,중립,0.60
3,F0UQQT0tSD8,@이영-z3g2h,국가위기상황 선포하고 중국인입국금지시키세요,2025-09-30T13:40:06Z,부정,0.90
4,F0UQQT0tSD8,@12SU15,해킹? 접속 아이피가311로 시작 되던데 311로 시작하는 아이피는 우리나라 아니지...,2025-09-29T22:16:42Z,중립,0.55


네이버 감정분석 시작

In [4]:
# ============================================================
# ✅ GPT 감정분석 초고속 안정 버전 (Batch + JSON 출력 + 캐싱 + 길이보정)  >  naver 뉴스 활용 
# ============================================================



import pandas as pd
import json, time, re
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """
    여러 문장을 batch 단위로 GPT에 전달하여 감정 분석 수행
    중복 문장은 캐시(cache)에서 불러옴
    """
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="GPT 감정분석 중"):
        batch = texts[i:i + batch_size]

        # ✅ 1. 캐시에 없는 문장만 추출
        new_texts = [t for t in batch if t not in cache]

        # ✅ 2. 모두 캐시에 있다면 바로 append
        if not new_texts:
            for t in batch:
                results.append(cache[t])
            continue

        # ✅ 3. GPT 프롬프트 (JSON 출력 강제)
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장들의 감정을 JSON 배열로 정확히 출력해.
        각 문장은 반드시 부정/중립/긍정 중 하나의 감정(sentiment)과 신뢰도(confidence, 0~1)를 포함해야 해.

        출력 형식 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # ⚡ 초고속 모델
                messages=[
                    {"role": "system", "content": "너는 감정 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()

            # ✅ 4. JSON 파싱 시도
            json_match = re.search(r"\[.*\]", output, re.S)
            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 백업 파서로 재시도")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            # ✅ 5. batch_results 길이 보정 (핵심 추가)
            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 시 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                batch_results = (batch_results + 
                                 [{"sentiment": "중립", "confidence": 0.0}] * (len(new_texts) - len(batch_results)))[:len(new_texts)]

            # ✅ 6. 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            # ✅ 7. 전체 결과 병합
            for t in batch:
                results.append(cache[t])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            for t in batch:
                results.append({"sentiment": "중립", "confidence": 0.0})

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 데이터 로드
# ------------------------------------------------------------
input_paths = [
    r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\GS_comments_2.csv",

]

# 파일 확장자별로 자동 판별해서 읽기
dfs = [
    pd.read_csv(path) if path.lower().endswith(".csv") else pd.read_excel(path)
    for path in input_paths
]
df = pd.concat(dfs, ignore_index=True)

if "comment" not in df.columns:
    raise ValueError("⚠️ 입력 파일에 'comment' 컬럼이 필요합니다!")

df["comment"] = df["comment"].astype(str).str.strip().str[:300]  # 과도한 길이 제한

# ------------------------------------------------------------
# 3️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["comment"].tolist(), batch_size=20)

df["gpt_sentiment"] = [r["sentiment"] for r in results]
df["gpt_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 4️⃣ 결과 저장 (엑셀)
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\GS_댓글감정분석_GPT_labeled.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ 분석 완료! 결과 저장 경로:\n📂 {output_path}")

# 상위 5행 미리보기
display(df.head())


GPT 감정분석 중: 100%|██████████| 30/30 [04:04<00:00,  8.14s/it]


✅ 분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\GS_댓글감정분석_GPT_labeled.xlsx


,channel,url,title,post_date,comment,gpt_sentiment,gpt_confidence
0,naver_news,https://n.news.naver.com/mnews/article/003/001...,NaN,2025.03.25. 12:55,개소리 말고 요기요나 다시 팔아버려라,부정,0.92
1,naver_news,https://n.news.naver.com/mnews/article/092/000...,NaN,2025.03.01. 13:32,개인정보 유출시키는 사람들을 법정 최고형 으로 다스려야 합니다.,긍정,0.85
2,naver_news,https://n.news.naver.com/mnews/article/092/000...,NaN,2025.03.01. 18:06,지들이 털려놓고 고객탓하긴,부정,0.88
3,naver_news,https://n.news.naver.com/mnews/article/092/000...,NaN,2025.03.01. 20:55,적게는 5개 많게는 수십개의 사이트 아이디랑 비번을 주기적으로 언제 다 바꾸냐......,부정,0.90
4,naver_news,https://n.news.naver.com/mnews/article/092/000...,NaN,2025.03.01. 19:51,해킹 당한 업체들은 과징금 크게 먹여라! 뭔 나라가 해킹에 이리 취약하냐?,부정,0.91


In [4]:
# ============================================================
# ✅ GPT 감정분석 초고속 안정 버전 (Batch + JSON 출력 + 캐싱 + 길이보정)  >  naver 뉴스 활용 
# ============================================================



import pandas as pd
import json, time, re
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """
    여러 문장을 batch 단위로 GPT에 전달하여 감정 분석 수행
    중복 문장은 캐시(cache)에서 불러옴
    """
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="GPT 감정분석 중"):
        batch = texts[i:i + batch_size]

        # ✅ 1. 캐시에 없는 문장만 추출
        new_texts = [t for t in batch if t not in cache]

        # ✅ 2. 모두 캐시에 있다면 바로 append
        if not new_texts:
            for t in batch:
                results.append(cache[t])
            continue

        # ✅ 3. GPT 프롬프트 (JSON 출력 강제)
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장들의 감정을 JSON 배열로 정확히 출력해.
        각 문장은 반드시 부정/중립/긍정 중 하나의 감정(sentiment)과 신뢰도(confidence, 0~1)를 포함해야 해.

        출력 형식 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # ⚡ 초고속 모델
                messages=[
                    {"role": "system", "content": "너는 감정 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()

            # ✅ 4. JSON 파싱 시도
            json_match = re.search(r"\[.*\]", output, re.S)
            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 백업 파서로 재시도")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            # ✅ 5. batch_results 길이 보정 (핵심 추가)
            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 시 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                batch_results = (batch_results + 
                                 [{"sentiment": "중립", "confidence": 0.0}] * (len(new_texts) - len(batch_results)))[:len(new_texts)]

            # ✅ 6. 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            # ✅ 7. 전체 결과 병합
            for t in batch:
                results.append(cache[t])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            for t in batch:
                results.append({"sentiment": "중립", "confidence": 0.0})

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 데이터 로드
# ------------------------------------------------------------
input_paths = [
    r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\GS_뉴스_20241223-20250406.csv",

]

# 파일 확장자별로 자동 판별해서 읽기
dfs = [
    pd.read_csv(path) if path.lower().endswith(".csv") else pd.read_excel(path)
    for path in input_paths
]
df = pd.concat(dfs, ignore_index=True)

if "뉴스제목" not in df.columns:
    raise ValueError("⚠️ 입력 파일에 'title' 컬럼이 필요합니다!")

df["뉴스제목"] = df["뉴스제목"].astype(str).str.strip().str[:300]  # 과도한 길이 제한

# ------------------------------------------------------------
# 3️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["뉴스제목"].tolist(), batch_size=20)

df["gpt_sentiment"] = [r["sentiment"] for r in results]
df["gpt_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 4️⃣ 결과 저장 (엑셀)
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\GS_뉴스감정분석_GPT_labeled.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ 분석 완료! 결과 저장 경로:\n📂 {output_path}")

# 상위 5행 미리보기
display(df.head())


GPT 감정분석 중: 100%|██████████| 10/10 [01:42<00:00, 10.29s/it]


✅ 분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\GS_뉴스감정분석_GPT_labeled.xlsx


,키워드,언론사,게시일,뉴스제목,url,gpt_sentiment,gpt_confidence
0,GS 해킹,경향신문,2025-04-02 21:18:21,비번 없이도 해킹·피싱 차단…‘패스키’로 눈돌리는 기업들,https://n.news.naver.com/mnews/article/032/000...,중립,0.65
1,GS 해킹,뉴스1,2025-03-23 09:59:00,"GS리테일, 52g 활동으로 고객·현장 중심 인공지능 전환 전개",https://n.news.naver.com/mnews/article/421/000...,긍정,0.70
2,GS 해킹,서울경제,2025-03-23 11:36:11,"GS리테일, 인공지능 전환 본격 전개",https://n.news.naver.com/mnews/article/011/000...,긍정,0.75
3,GS 해킹,매일신문,2025-03-23 13:24:16,"""고객 불편 AI로 해소"" GS리테일, '현장 중심 AX' 선언",https://n.news.naver.com/mnews/article/088/000...,긍정,0.80
4,GS 해킹,뉴시스,2025-03-24 09:17:43,"GS리테일, 52g 활동으로 고객과 현장 중심 AI 전환 본격 전개",https://n.news.naver.com/mnews/article/003/001...,긍정,0.75


In [5]:
# ============================================================
# ✅ GPT 감정분석 초고속 안정 버전 (Batch + JSON 출력 + 캐싱 + 길이보정)  >  naver 뉴스 활용 
# ============================================================



import pandas as pd
import json, time, re
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """
    여러 문장을 batch 단위로 GPT에 전달하여 감정 분석 수행
    중복 문장은 캐시(cache)에서 불러옴
    """
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="GPT 감정분석 중"):
        batch = texts[i:i + batch_size]

        # ✅ 1. 캐시에 없는 문장만 추출
        new_texts = [t for t in batch if t not in cache]

        # ✅ 2. 모두 캐시에 있다면 바로 append
        if not new_texts:
            for t in batch:
                results.append(cache[t])
            continue

        # ✅ 3. GPT 프롬프트 (JSON 출력 강제)
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장들의 감정을 JSON 배열로 정확히 출력해.
        각 문장은 반드시 부정/중립/긍정 중 하나의 감정(sentiment)과 신뢰도(confidence, 0~1)를 포함해야 해.

        출력 형식 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # ⚡ 초고속 모델
                messages=[
                    {"role": "system", "content": "너는 감정 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()

            # ✅ 4. JSON 파싱 시도
            json_match = re.search(r"\[.*\]", output, re.S)
            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 백업 파서로 재시도")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            # ✅ 5. batch_results 길이 보정 (핵심 추가)
            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 시 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                batch_results = (batch_results + 
                                 [{"sentiment": "중립", "confidence": 0.0}] * (len(new_texts) - len(batch_results)))[:len(new_texts)]

            # ✅ 6. 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            # ✅ 7. 전체 결과 병합
            for t in batch:
                results.append(cache[t])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            for t in batch:
                results.append({"sentiment": "중립", "confidence": 0.0})

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 데이터 로드
# ------------------------------------------------------------
input_paths = [
    r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\KT_comments_2.csv",


]

# 파일 확장자별로 자동 판별해서 읽기
dfs = [
    pd.read_csv(path) if path.lower().endswith(".csv") else pd.read_excel(path)
    for path in input_paths
]
df = pd.concat(dfs, ignore_index=True)

if "comment" not in df.columns:
    raise ValueError("⚠️ 입력 파일에 'comment' 컬럼이 필요합니다!")

df["comment"] = df["comment"].astype(str).str.strip().str[:300]  # 과도한 길이 제한

# ------------------------------------------------------------
# 3️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["comment"].tolist(), batch_size=20)

df["gpt_sentiment"] = [r["sentiment"] for r in results]
df["gpt_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 4️⃣ 결과 저장 (엑셀)
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\KT_댓글글감정분석_GPT_labeled.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ 분석 완료! 결과 저장 경로:\n📂 {output_path}")

# 상위 5행 미리보기
display(df.head())


GPT 감정분석 중: 100%|██████████| 113/113 [27:53<00:00, 14.81s/it]  



✅ 분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\KT_댓글글감정분석_GPT_labeled.xlsx


,channel,url,title,post_date,comment,gpt_sentiment,gpt_confidence
0,naver_news,https://n.news.naver.com/mnews/article/032/000...,NaN,2025.10.28. 14:44,어차피 터질일 아니었나? KT 가입하면 여기저기서 뭐 가입해라 뭐 가입해라 전화오는...,부정,0.95
1,naver_news,https://n.news.naver.com/mnews/article/032/000...,NaN,2025.10.28. 14:54,skt가 그나마 나은 거 였네. 3사 다 털림,긍정,0.80
2,naver_news,https://n.news.naver.com/mnews/article/032/000...,NaN,2025.10.28. 14:30,"징벌적 손해배상적용으로 정보유출된 개인에게 피해금액이 없더라도, 1인당 10만원씩 ...",긍정,0.85
3,naver_news,https://n.news.naver.com/mnews/article/032/000...,NaN,2025.10.29. 08:56,은폐를 했다는게 가장큰 문제 가입자 스스로 판단할수 있도록 위약금 전 고객면제해야함,부정,0.90
4,naver_news,https://n.news.naver.com/mnews/article/032/000...,NaN,2025.10.28. 14:57,3사가 똑 같은 상황임.. 정부 관리부실로 데이타 센터 화재 난 것은 피해보상 없냐 ??,부정,0.88


In [6]:
# ============================================================
# ✅ GPT 감정분석 초고속 안정 버전 (Batch + JSON 출력 + 캐싱 + 길이보정)  >  naver 뉴스 활용 
# ============================================================



import pandas as pd
import json, time, re
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """
    여러 문장을 batch 단위로 GPT에 전달하여 감정 분석 수행
    중복 문장은 캐시(cache)에서 불러옴
    """
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="GPT 감정분석 중"):
        batch = texts[i:i + batch_size]

        # ✅ 1. 캐시에 없는 문장만 추출
        new_texts = [t for t in batch if t not in cache]

        # ✅ 2. 모두 캐시에 있다면 바로 append
        if not new_texts:
            for t in batch:
                results.append(cache[t])
            continue

        # ✅ 3. GPT 프롬프트 (JSON 출력 강제)
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장들의 감정을 JSON 배열로 정확히 출력해.
        각 문장은 반드시 부정/중립/긍정 중 하나의 감정(sentiment)과 신뢰도(confidence, 0~1)를 포함해야 해.

        출력 형식 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # ⚡ 초고속 모델
                messages=[
                    {"role": "system", "content": "너는 감정 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()

            # ✅ 4. JSON 파싱 시도
            json_match = re.search(r"\[.*\]", output, re.S)
            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 백업 파서로 재시도")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            # ✅ 5. batch_results 길이 보정 (핵심 추가)
            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 시 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                batch_results = (batch_results + 
                                 [{"sentiment": "중립", "confidence": 0.0}] * (len(new_texts) - len(batch_results)))[:len(new_texts)]

            # ✅ 6. 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            # ✅ 7. 전체 결과 병합
            for t in batch:
                results.append(cache[t])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            for t in batch:
                results.append({"sentiment": "중립", "confidence": 0.0})

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 데이터 로드
# ------------------------------------------------------------
input_paths = [
    r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\KT_뉴스_20250901-20251031.csv",

]

# 파일 확장자별로 자동 판별해서 읽기
dfs = [
    pd.read_csv(path) if path.lower().endswith(".csv") else pd.read_excel(path)
    for path in input_paths
]
df = pd.concat(dfs, ignore_index=True)

if "뉴스제목" not in df.columns:
    raise ValueError("⚠️ 입력 파일에 'comment' 컬럼이 필요합니다!")

df["뉴스제목"] = df["뉴스제목"].astype(str).str.strip().str[:300]  # 과도한 길이 제한

# ------------------------------------------------------------
# 3️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["뉴스제목"].tolist(), batch_size=20)

df["gpt_sentiment"] = [r["sentiment"] for r in results]
df["gpt_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 4️⃣ 결과 저장 (엑셀)
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\KT_뉴스감정분석_GPT_labeled.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ 분석 완료! 결과 저장 경로:\n📂 {output_path}")

# 상위 5행 미리보기
display(df.head())


GPT 감정분석 중: 100%|██████████| 22/22 [03:42<00:00, 10.09s/it]


✅ 분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\KT_뉴스감정분석_GPT_labeled.xlsx


,키워드,언론사,게시일,뉴스제목,url,gpt_sentiment,gpt_confidence
0,KT 펨토셀 해킹,이데일리,2025-10-29 16:02:09,"김영섭 KT 대표 “전체 고객 위약금 면제, 종합 검토할 것”",https://n.news.naver.com/mnews/article/018/000...,중립,0.65
1,KT 펨토셀 해킹,블로터,2025-10-29 17:36:14,"KT 김영섭, 국감서 연임 도전 여부 질문에 ""이사회서 입장 낼 것""",https://n.news.naver.com/mnews/article/293/000...,중립,0.60
2,KT 펨토셀 해킹,서울경제,2025-10-29 15:18:17,"KT, 소액결제 피해자 위약금 면제…月 100GB 무료",https://n.news.naver.com/mnews/article/011/000...,긍정,0.85
3,KT 펨토셀 해킹,디지털타임스,2025-10-29 15:32:49,KT “소액결제 피해 고객에 100GB·15만원 요금할인 제공”,https://n.news.naver.com/mnews/article/029/000...,긍정,0.80
4,KT 펨토셀 해킹,경향신문,2025-10-28 14:04:00,KT 이용자 10명 중 8명 “전 고객 대상 위약금 면제해야”,https://n.news.naver.com/mnews/article/032/000...,긍정,0.75


In [7]:
# ============================================================
# ✅ GPT 감정분석 초고속 안정 버전 (Batch + JSON 출력 + 캐싱 + 길이보정)  >  naver 뉴스 활용 
# ============================================================



import pandas as pd
import json, time, re
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """
    여러 문장을 batch 단위로 GPT에 전달하여 감정 분석 수행
    중복 문장은 캐시(cache)에서 불러옴
    """
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="GPT 감정분석 중"):
        batch = texts[i:i + batch_size]

        # ✅ 1. 캐시에 없는 문장만 추출
        new_texts = [t for t in batch if t not in cache]

        # ✅ 2. 모두 캐시에 있다면 바로 append
        if not new_texts:
            for t in batch:
                results.append(cache[t])
            continue

        # ✅ 3. GPT 프롬프트 (JSON 출력 강제)
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장들의 감정을 JSON 배열로 정확히 출력해.
        각 문장은 반드시 부정/중립/긍정 중 하나의 감정(sentiment)과 신뢰도(confidence, 0~1)를 포함해야 해.

        출력 형식 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # ⚡ 초고속 모델
                messages=[
                    {"role": "system", "content": "너는 감정 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()

            # ✅ 4. JSON 파싱 시도
            json_match = re.search(r"\[.*\]", output, re.S)
            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 백업 파서로 재시도")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            # ✅ 5. batch_results 길이 보정 (핵심 추가)
            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 시 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                batch_results = (batch_results + 
                                 [{"sentiment": "중립", "confidence": 0.0}] * (len(new_texts) - len(batch_results)))[:len(new_texts)]

            # ✅ 6. 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            # ✅ 7. 전체 결과 병합
            for t in batch:
                results.append(cache[t])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            for t in batch:
                results.append({"sentiment": "중립", "confidence": 0.0})

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 데이터 로드
# ------------------------------------------------------------
input_paths = [
    r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\lotte_comments_2.csv",


]

# 파일 확장자별로 자동 판별해서 읽기
dfs = [
    pd.read_csv(path) if path.lower().endswith(".csv") else pd.read_excel(path)
    for path in input_paths
]
df = pd.concat(dfs, ignore_index=True)

if "comment" not in df.columns:
    raise ValueError("⚠️ 입력 파일에 'comment' 컬럼이 필요합니다!")

df["comment"] = df["comment"].astype(str).str.strip().str[:300]  # 과도한 길이 제한

# ------------------------------------------------------------
# 3️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["comment"].tolist(), batch_size=20)

df["gpt_sentiment"] = [r["sentiment"] for r in results]
df["gpt_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 4️⃣ 결과 저장 (엑셀)
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\롯데_댓글감정분석_GPT_labeled.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ 분석 완료! 결과 저장 경로:\n📂 {output_path}")

# 상위 5행 미리보기
display(df.head())


GPT 감정분석 중: 100%|██████████| 115/115 [15:37<00:00,  8.15s/it]



✅ 분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\롯데_댓글감정분석_GPT_labeled.xlsx


,channel,url,title,post_date,comment,gpt_sentiment,gpt_confidence
0,naver_news,https://n.news.naver.com/mnews/article/079/000...,NaN,2025.10.13. 13:50,"장난치나. 주민번호까지 털렸는데, 보호조치 같은 소리하고 있네. 어디 소송하는 곳 없나?",부정,0.95
1,naver_news,https://n.news.naver.com/mnews/article/366/000...,NaN,2025.10.14. 17:55,클린봇이 부적절한 표현을 감지한 댓글입니다.,중립,0.60
2,naver_news,https://n.news.naver.com/mnews/article/586/000...,NaN,2025.10.13. 12:42,접대 11번 받으러 간거네 기업 유흥 법카의 원흉,부정,0.90
3,naver_news,https://n.news.naver.com/mnews/article/003/001...,NaN,2025.10.14. 15:08,국개의원넘아 검사를 했던 금감원한테 뭐라해야지 말대로 걔네가 검사를 소홀이 한거잖아,부정,0.92
4,naver_news,https://n.news.naver.com/mnews/article/003/001...,NaN,2025.10.16. 11:33,심사기간이 너무 짧습니다. 그래서 현장 점검할 시간이 거의 없다고 보면 됩니다. 서...,부정,0.85


In [8]:
# ============================================================
# ✅ GPT 감정분석 초고속 안정 버전 (Batch + JSON 출력 + 캐싱 + 길이보정)  >  naver 뉴스 활용 
# ============================================================



import pandas as pd
import json, time, re
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """
    여러 문장을 batch 단위로 GPT에 전달하여 감정 분석 수행
    중복 문장은 캐시(cache)에서 불러옴
    """
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="GPT 감정분석 중"):
        batch = texts[i:i + batch_size]

        # ✅ 1. 캐시에 없는 문장만 추출
        new_texts = [t for t in batch if t not in cache]

        # ✅ 2. 모두 캐시에 있다면 바로 append
        if not new_texts:
            for t in batch:
                results.append(cache[t])
            continue

        # ✅ 3. GPT 프롬프트 (JSON 출력 강제)
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장들의 감정을 JSON 배열로 정확히 출력해.
        각 문장은 반드시 부정/중립/긍정 중 하나의 감정(sentiment)과 신뢰도(confidence, 0~1)를 포함해야 해.

        출력 형식 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # ⚡ 초고속 모델
                messages=[
                    {"role": "system", "content": "너는 감정 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()

            # ✅ 4. JSON 파싱 시도
            json_match = re.search(r"\[.*\]", output, re.S)
            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 백업 파서로 재시도")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            # ✅ 5. batch_results 길이 보정 (핵심 추가)
            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 시 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                batch_results = (batch_results + 
                                 [{"sentiment": "중립", "confidence": 0.0}] * (len(new_texts) - len(batch_results)))[:len(new_texts)]

            # ✅ 6. 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            # ✅ 7. 전체 결과 병합
            for t in batch:
                results.append(cache[t])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            for t in batch:
                results.append({"sentiment": "중립", "confidence": 0.0})

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 데이터 로드
# ------------------------------------------------------------
input_paths = [
    r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\롯데카드_뉴스_20250819-20251016.csv",
 


]

# 파일 확장자별로 자동 판별해서 읽기
dfs = [
    pd.read_csv(path) if path.lower().endswith(".csv") else pd.read_excel(path)
    for path in input_paths
]
df = pd.concat(dfs, ignore_index=True)

if "뉴스제목" not in df.columns:
    raise ValueError("⚠️ 입력 파일에 'comment' 컬럼이 필요합니다!")

df["뉴스제목"] = df["뉴스제목"].astype(str).str.strip().str[:300]  # 과도한 길이 제한

# ------------------------------------------------------------
# 3️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["뉴스제목"].tolist(), batch_size=20)

df["gpt_sentiment"] = [r["sentiment"] for r in results]
df["gpt_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 4️⃣ 결과 저장 (엑셀)
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\롯데_뉴스글감정분석_GPT_labeled.xlsx"
df.to_excel(output_path, index=False)
print(f"\n✅ 분석 완료! 결과 저장 경로:\n📂 {output_path}")

# 상위 5행 미리보기
display(df.head())


GPT 감정분석 중: 100%|██████████| 17/17 [02:42<00:00,  9.53s/it]


✅ 분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\롯데_뉴스글감정분석_GPT_labeled.xlsx


,키워드,언론사,게시일,뉴스제목,url,gpt_sentiment,gpt_confidence
0,롯데 정보유출,미디어오늘,2025-10-16 21:05:09,개인정보위원회 수장에 AI 전문가 임명…시민사회 우려,https://n.news.naver.com/mnews/article/006/000...,부정,0.85
1,롯데 정보유출,연합뉴스,2025-10-16 09:14:01,"KT·롯데카드 해킹 사태인데…KISA, 제주 워크숍 논란",https://n.news.naver.com/mnews/article/001/001...,부정,0.88
2,롯데 정보유출,조선비즈,2025-10-16 12:44:14,"“KT·롯데카드 해킹 사태 진행 중인데”… KISA, 제주 워크숍 강행 논란",https://n.news.naver.com/mnews/article/366/000...,부정,0.87
3,롯데 정보유출,MBC,2025-10-13 11:38:15,"롯데카드 해킹 두 달째, 회원 탈퇴 3만 명·카드 해지 5만 명",https://n.news.naver.com/mnews/article/214/000...,부정,0.90
4,롯데 정보유출,노컷뉴스,2025-10-13 13:00:11,"롯데카드 ""민감정보 유출 고객 82% 보호조치 완료""",https://n.news.naver.com/mnews/article/079/000...,긍정,0.75


In [10]:
# ============================================================
# ✅ GPT 감정분석 초고속 안정 버전 (Batch + JSON 출력 + 캐싱 + 길이보정)  >  naver 뉴스 활용 
# ============================================================

import pandas as pd
import json, time, re, os
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """
    여러 문장을 batch 단위로 GPT에 전달하여 감정 분석 수행
    중복 문장은 캐시(cache)에서 불러옴
    """
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="GPT 감정분석 중"):
        batch = texts[i:i + batch_size]

        # ✅ 1. 캐시에 없는 문장만 추출
        new_texts = [t for t in batch if t not in cache]

        # ✅ 2. 모두 캐시에 있다면 바로 append
        if not new_texts:
            for t in batch:
                results.append(cache[t])
            continue

        # ✅ 3. GPT 프롬프트 (JSON 출력 강제)
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장들의 감정을 JSON 배열로 정확히 출력해.
        각 문장은 반드시 부정/중립/긍정 중 하나의 감정(sentiment)과 신뢰도(confidence, 0~1)를 포함해야 해.

        출력 형식 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # ⚡ 초고속 모델
                messages=[
                    {"role": "system", "content": "너는 감정 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()

            # ✅ 4. JSON 파싱 시도
            json_match = re.search(r"\[.*\]", output, re.S)
            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 백업 파서로 재시도")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            # ✅ 5. batch_results 길이 보정 (핵심 추가)
            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 시 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                batch_results = (batch_results + 
                                 [{"sentiment": "중립", "confidence": 0.0}] * (len(new_texts) - len(batch_results)))[:len(new_texts)]

            # ✅ 6. 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            # ✅ 7. 전체 결과 병합
            for t in batch:
                results.append(cache[t])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            for t in batch:
                results.append({"sentiment": "중립", "confidence": 0.0})

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 데이터 로드
# ------------------------------------------------------------
input_paths = [
    r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\SKT_comments_2.csv",
]

# 파일 확장자별로 자동 판별해서 읽기
dfs = []
for path in input_paths:
    path = path.strip()
    # 파일이 csv인지 xlsx/xls 인지 판단
    _, ext = os.path.splitext(path)
    ext = ext.lower()
    if ext == ".csv":
        dfs.append(pd.read_csv(path, encoding="utf-8"))
    elif ext in [".xlsx", ".xls"]:
        # 엑셀 파일은 openpyxl 엔진 지정
        dfs.append(pd.read_excel(path, engine="openpyxl"))
    else:
        raise ValueError(f"지원하지 않는 파일 형식입니다: {ext}")

df = pd.concat(dfs, ignore_index=True)

if "comment" not in df.columns:
    raise ValueError("⚠️ 입력 파일에 'comment' 컬럼이 필요합니다!")

df["comment"] = df["comment"].astype(str).str.strip().str[:300]  # 과도한 길이 제한

# ------------------------------------------------------------
# 3️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["comment"].tolist(), batch_size=20)

df["gpt_sentiment"] = [r["sentiment"] for r in results]
df["gpt_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 4️⃣ 결과 저장 (엑셀)
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\SKT_댓글감정분석_GPT_labeled.xlsx"
# 엑셀 저장 시 openpyxl 엔진 지정
df.to_excel(output_path, index=False, engine="openpyxl")
print(f"\n✅ 분석 완료! 결과 저장 경로:\n📂 {output_path}")

# 상위 5행 미리보기
display(df.head())


GPT 감정분석 중: 100%|██████████| 770/770 [1:48:16<00:00,  8.44s/it]



✅ 분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\SKT_댓글감정분석_GPT_labeled.xlsx


,channel,url,title,post_date,comment,gpt_sentiment,gpt_confidence
0,naver_news,https://n.news.naver.com/mnews/article/417/000...,NaN,2025.05.30. 21:38,난 왜 안바꿔주냐,부정,0.85
1,naver_news,https://n.news.naver.com/mnews/article/417/000...,NaN,2025.05.31. 01:24,다음달쯤 유심 바꾸고 번호이동해야지,중립,0.60
2,naver_news,https://n.news.naver.com/mnews/article/417/000...,NaN,2025.05.30. 21:34,난 아직,중립,0.50
3,naver_news,https://n.news.naver.com/mnews/article/417/000...,NaN,2025.05.30. 21:29,기업이미지 이미 상실됨 기지국변경 현실화,부정,0.90
4,naver_news,https://n.news.naver.com/mnews/article/417/000...,NaN,2025.05.30. 15:18,아직도 그회사를 왜쓰는거지?,부정,0.88


In [12]:
# ============================================================
# ✅ GPT 감정분석 초고속 안정 버전 (Batch + JSON 출력 + 캐싱 + 길이보정)
#    - Naver 뉴스 감정 분석용
# ============================================================

import pandas as pd
import json, time, re
from openai import OpenAI
from tqdm import tqdm

# 🔑 OpenAI API 키 설정
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱 + 길이보정)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="GPT 감정분석 중"):
        batch = texts[i:i + batch_size]
        new_texts = [t for t in batch if t not in cache]

        if not new_texts:
            results.extend([cache[t] for t in batch])
            continue

        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장들의 감정을 JSON 배열로 정확히 출력해.
        각 문장은 반드시 부정/중립/긍정 중 하나의 감정(sentiment)과 신뢰도(confidence, 0~1)를 포함해야 해.

        출력 형식 예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.93}},
          {{"id":2, "sentiment":"중립", "confidence":0.55}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "너는 감정 분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()
            json_match = re.search(r"\[.*\]", output, re.S)

            if json_match:
                try:
                    parsed = json.loads(json_match.group(0))
                except json.JSONDecodeError:
                    print("⚠️ JSON Decode Error → 빈 리스트로 대체")
                    parsed = []
            else:
                print("⚠️ JSON 파싱 실패, 원문:", output[:200])
                parsed = []

            batch_results = []
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                    batch_results.append({"sentiment": s, "confidence": c})
                else:
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})

            # 🔒 길이 불일치 자동 보정
            if len(batch_results) != len(new_texts):
                print(f"⚠️ Batch 길이 불일치: 입력 {len(new_texts)}, 결과 {len(batch_results)} → 자동보정")
                while len(batch_results) < len(new_texts):
                    batch_results.append({"sentiment": "중립", "confidence": 0.0})
                batch_results = batch_results[:len(new_texts)]

            # ✅ 캐시에 저장
            for t, r in zip(new_texts, batch_results):
                cache[t] = r

            results.extend([cache[t] for t in batch])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            results.extend([{"sentiment": "중립", "confidence": 0.0} for _ in batch])

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 데이터 로드 (파일 확장자 자동 판별 + 공백 제거 + 인코딩 처리)
# ------------------------------------------------------------
input_paths = [
    r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\SKT_뉴스_20250401-20250531.csv",
]

dfs = []
for path in input_paths:
    path = path.strip()
    try:
        if path.lower().endswith(".csv"):
            df_temp = pd.read_csv(path, encoding="utf-8-sig")
        elif path.lower().endswith((".xls", ".xlsx")):
            df_temp = pd.read_excel(path, engine="openpyxl")
        else:
            raise ValueError(f"지원하지 않는 파일 형식: {path}")
        dfs.append(df_temp)
    except Exception as e:
        print(f"⚠️ 파일 로드 실패: {path} → {e}")

df = pd.concat(dfs, ignore_index=True)

if "뉴스제목" not in df.columns:
    raise ValueError("⚠️ 입력 파일에 '뉴스제목' 컬럼이 필요합니다!")

df["뉴스제목"] = df["뉴스제목"].astype(str).str.strip().str[:300]

# ------------------------------------------------------------
# 3️⃣ GPT 감정분석 실행
# ------------------------------------------------------------
results = gpt_batch_sentiment(df["뉴스제목"].tolist(), batch_size=20)

df["gpt_sentiment"] = [r["sentiment"] for r in results]
df["gpt_confidence"] = [r["confidence"] for r in results]

# ------------------------------------------------------------
# 4️⃣ 결과 저장
# ------------------------------------------------------------
output_path = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\SKT_뉴스감정분석_GPT_labeled.xlsx"
df.to_excel(output_path, index=False, engine="openpyxl")

print(f"\n✅ 분석 완료! 결과 저장 경로:\n📂 {output_path}")
display(df.head())


GPT 감정분석 중: 100%|██████████| 45/45 [07:15<00:00,  9.69s/it]


✅ 분석 완료! 결과 저장 경로:
📂 C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\SKT_뉴스감정분석_GPT_labeled.xlsx


,키워드,언론사,게시일,뉴스제목,url,gpt_sentiment,gpt_confidence
0,SK 유심 해킹,머니S,2025-05-30 14:58:14,유심 교체 SK텔레콤 고객 537만명… 대기는 372만명,https://n.news.naver.com/mnews/article/417/000...,중립,0.85
1,SK 유심 해킹,뉴시스,2025-05-30 11:01:52,"SKT, 가입자 537만명 '유심 교체'…예약자 372만명 남아",https://n.news.naver.com/mnews/article/003/001...,중립,0.85
2,SK 유심 해킹,이데일리,2025-05-30 12:16:21,"SK텔레콤, 유심교체 537만건…예약대기 372만명",https://n.news.naver.com/mnews/article/018/000...,중립,0.85
3,SK 유심 해킹,부산일보,2025-05-30 14:58:13,SKT “가입자 537만 명 유심 교체”…잔여 예약자 372만 명,https://n.news.naver.com/mnews/article/082/000...,중립,0.85
4,SK 유심 해킹,뉴시스,2025-05-30 06:00:00,SKT 40% 점유율 붕괴 초읽기…가입자 40만 잃었다,https://n.news.naver.com/mnews/article/003/001...,부정,0.92
